<a href="https://colab.research.google.com/github/mahdimalverdi/BigData-Exam/blob/main/Exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()

In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
spark = SparkSession.builder.master("local[1]").appName("hw6").getOrCreate()

In [6]:
!wget https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/data.csv

--2021-06-22 11:00:25--  https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64409432 (61M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  61.42M   117MB/s    in 0.5s    

2021-06-22 11:00:27 (117 MB/s) - ‘data.csv’ saved [64409432/64409432]



In [16]:
!wget  https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/labels.csv

--2021-06-22 11:06:02--  https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1112 (1.1K) [text/plain]
Saving to: ‘labels.csv’

labels.csv          100%[===================>]   1.09K  --.-KB/s    in 0s      

2021-06-22 11:06:03 (46.7 MB/s) - ‘labels.csv’ saved [1112/1112]



In [11]:
train_dataset = spark.read.option("maxColumns", 50_000).csv("data.csv",inferSchema=True, header =False)
train_dataset.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tru

In [17]:
lable_dataset = spark.read.option("maxColumns", 50_000).csv("labels.csv",inferSchema=True, header =True)
lable_dataset.printSchema()

root
 |-- lable: integer (nullable = true)



In [ ]:
from pyspark.sql.types import StructType, StructField, LongType

def with_column_index(sdf): 
    new_schema = StructType(sdf.schema.fields + [StructField("ColumnIndex", LongType(), False),])
    return sdf.rdd.zipWithIndex().map(lambda row: row[0] + (row[1],)).toDF(schema=new_schema)

df1_ci = with_column_index(lable_dataset)
df2_ci = with_column_index(train_dataset)
join_on_index = df1_ci.join(df2_ci, df1_ci.ColumnIndex == df2_ci.ColumnIndex, 'inner').drop("ColumnIndex")

In [ ]:
def getHeaders(n):
  return n.name

headers = list(map(getHeaders,train_dataset.schema.fields))

In [ ]:
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as F

assembler = VectorAssembler(inputCols=headers,outputCol = 'Attributes')

output = assembler.transform(train_dataset)
finalized_data = output.select("Attributes")